In [ ]:
!pip install pydub language-tool-python librosa transformers

In [ ]:
!pip install pytube ffmpeg openai

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 3.7 MB/s eta 0:00:00
  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6082 sha256=c26a6afa164c11fe72916cb6b016c4aac3917ad7c1dd9d92c34289a49f1c39a1
  Stored in directory: /root/.cache/pip/wheels/8e/7a/69/cd6aeb83b126a7f04cbe7c9d929028dc52a6e7d525ff56003a
Successfully built ffmpeg


In [ ]:
!pip install google-cloud-speech

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.0/305.0 kB 16.1 MB/s eta 0:00:00


In [ ]:
import os
import io
from google.cloud import speech

In [ ]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS']="credentials"

In [ ]:
!pip install phonemizer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 11.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.8/103.8 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.4/213.4 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 562.4/562.4 kB 30.8 MB/s eta 0:00:00
  Created wheel for pylatexenc: filename=pylatexenc-2.10-py3-none-any.whl size=136816 sha256=07f925864e2ba0d73c46dcb81b27864214fc27ec9a733d9c5e310558d1277c1d
  Stored in directory: /root/.cache/pip/wheels/d3/31/8b/e09b0386afd80cfc556c00408c9aeea5c35c4d484a9c762fd5
Successfully built pylatexenc


In [ ]:
!sudo apt-get update
!sudo apt-get install espeak-ng -y

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,458 kB]
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,196 kB]
Get:13 https://r2u.stat.illinois.edu/ubunt

In [ ]:
import google.generativeai as genai

genai.configure(api_key="api_key")
language_model = genai.GenerativeModel("gemini-pro")

In [ ]:
import os
import torch
from pydub import AudioSegment, silence
import language_tool_python as lt
import librosa
import numpy as np
from openai import OpenAI
from phonemizer import phonemize
import string


input_audio_path = "/content/Recording.wav"
temp_audio_path = "resampled.wav"

import soundfile as sf

def preprocess_audio(file_path, target_sr=16000):
    y, sr = librosa.load(file_path, sr=None)
    if sr != target_sr:
        y = librosa.resample(y, orig_sr=sr, target_sr=target_sr)
    sf.write(temp_audio_path, y, target_sr, subtype="PCM_16")
    return temp_audio_path, target_sr

def transcribe_audio(audio_path):

    client = speech.SpeechClient()


    with open(audio_path, "rb") as audio_file:
        content = audio_file.read()

    audio = speech.RecognitionAudio(content=content)


    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
        language_code="en-IN",
        enable_automatic_punctuation=True,
    )


    response = client.recognize(config=config, audio=audio)


    transcription = " ".join([result.alternatives[0].transcript for result in response.results])
    return transcription
def grammar_analysis(text):
    prompt = f"""
    You are an English grammar expert. You will be provided with the response of a candidate.
    Your task is to parse the response and generate the number of grammatical errors, evaluating the
    candidate's grammatical correctness. Avoid considering punctuation marks because this is transcribed.
    Candidate's Response:
    {text}
    Please provide the number of grammatical errors without any explanation.
    """

    response = model.generate_content(prompt)
    grammar_errors = response.text
    return int(grammar_errors)
def text_to_phonemes(text):
    phonemes = phonemize(text, language="en-us", backend="espeak")
    return phonemes.split()
def pronunciation_analysis(transcribed_text, original_audio_path):
    word_count = len(transcribed_text.split())
    audio_duration = librosa.get_duration(filename=original_audio_path)
    speaking_rate = word_count / audio_duration
    return word_count, audio_duration, speaking_rate

def fluency_analysis(transcribed_text, audio_path):
    audio = AudioSegment.from_file(audio_path)
    silences = silence.detect_silence(audio, min_silence_len=2000, silence_thresh=-40)
    print(silences)
    num_pauses = len(silences)
    filler_words = ["um", "uh", "like", "so", "actually", "basically", "literally", "you know"]
    words = transcribed_text.translate(str.maketrans('', '', string.punctuation)).split()
    detected_fillers = []
    for i in range(len(words)):
        if words[i].lower() in filler_words:
            detected_fillers.append(words[i])
        if i < len(words) - 1 and f"{words[i].lower()} {words[i+1].lower()}" == "you know":
            detected_fillers.append("you know")
    return num_pauses, detected_fillers


print("Preprocessing audio...")
processed_path, sr = preprocess_audio(input_audio_path)

print("Transcribing audio...")
transcription = transcribe_audio(processed_path)
print(f"Transcription: {transcription}")

print("Evaluating grammar...")
grammar_errors = grammar_analysis(transcription)
print(f"Grammar Errors: {grammar_errors}")

print("Evaluating pronunciation...")
expected_phonemes = text_to_phonemes(transcription)
print(f"Phonemes: {expected_phonemes}")

print("Evaluating fluency...")
num_pauses, filler_words = fluency_analysis(transcription, processed_path)
word_count, duration, speaking_rate = pronunciation_analysis(transcription, processed_path)
print(f"Word Count: {word_count}")
print(f"Audio Duration: {duration:.2f} seconds")
print(f"Speaking Rate: {speaking_rate:.2f} words per second")
print(f"Number of Pauses: {num_pauses}")
print(f"Filler Words Detected: {len(filler_words)}")

print("\n### Final Assessment ###")
print(f"Transcription: {transcription}")
print(f"Grammar Errors:{grammar_errors}")
print(f"Word Count: {word_count}, Speaking Rate: {speaking_rate:.2f} WPS")
print(f"Number of Pauses: {num_pauses}")
print(f"Phonemes: {expected_phonemes}")
print(f"Filler Words Detected: {filler_words}")


if os.path.exists(temp_audio_path):
    os.remove(temp_audio_path)



Preprocessing audio...
Transcribing audio...


DefaultCredentialsError: File pluginlive-444306-d4f6569e136a.json was not found.

In [ ]:
words= transcription.split(' ')
num_words=len(words)

#generating grammar score
grammar_mistake = 1- (grammar_errors/num_words)
grammar_score= 5*grammar_mistake
grammar_score=round(grammar_score,2)

#generating fluency score
fluency_mistake = 1- ((num_pauses+len(filler_words))/num_words)
fluency_score=5*fluency_mistake
fluency_score=round(fluency_score, 2)

print(f"Grammar Score: {grammar_score}")
print(f"Fluency Score: {fluency_score}")

def generate_positive_feedback(grammar_score, fluency_score):
    feedback = {}

    # Grammar feedback
    if grammar_score >= 4.90:
        feedback['grammar'] = "Your grammar is outstanding, with exceptional clarity and no noticeable errors. Keep up the excellent work!"
    elif grammar_score >= 4.75:
        feedback['grammar'] = "Your grammar is excellent, with very few, nearly unnoticeable errors. A little more attention to detail will make it flawless."
    elif grammar_score >= 4.60:
        feedback['grammar'] = "Your grammar is strong, with only occasional minor errors. With consistent refinement, you can reach perfection."
    elif grammar_score >= 4.45:
        feedback['grammar'] = "Your grammar is very good, with a few minor areas for improvement. Regular practice will help you polish these aspects."
    elif grammar_score >= 4.30:
        feedback['grammar'] = "Your grammar is good and clear, though some noticeable errors exist. Focused practice can enhance your clarity even further."
    elif grammar_score >= 4.00:
        feedback['grammar'] = "Your grammar is adequate and generally understandable, but there is room for improvement in clarity and precision. Keep refining your skills."
    elif grammar_score >= 3.75:
        feedback['grammar'] = "Your grammar is improving, though noticeable errors are present. Continued practice and attention to detail will help you progress."
    elif grammar_score >= 3.50:
        feedback['grammar'] = "Your grammar shows potential, but frequent errors can distract from clarity. Focused effort on common patterns can significantly enhance your accuracy."
    else:
        feedback['grammar'] = "Your grammar is developing, but it requires consistent attention. Identify frequent errors and work on them to strengthen your communication skills."

    # Fluency feedback
    if fluency_score >= 4.90:
        feedback['fluency'] = "Your fluency is remarkable, with an effortless and natural flow. It is a pleasure to listen to!"
    elif fluency_score >= 4.75:
        feedback['fluency'] = "Your fluency is very strong, with rare and minor hesitations. Continued practice will make it even more seamless."
    elif fluency_score >= 4.60:
        feedback['fluency'] = "Your fluency is strong, with only slight hesitations. Practicing in real-time settings will make your speech even more fluid."
    elif fluency_score >= 4.45:
        feedback['fluency'] = "Your fluency is good, with occasional hesitations. Regular speaking exercises will help you improve your confidence and flow."
    elif fluency_score >= 4.30:
        feedback['fluency'] = "Your fluency is moderate, with room for improvement. Engaging in more speaking opportunities will help make your speech more natural."
    elif fluency_score >= 4.00:
        feedback['fluency'] = "Your fluency is adequate but could be smoother. Focused exercises in spontaneous speaking will enhance your confidence."
    elif fluency_score >= 3.75:
        feedback['fluency'] = "Your fluency is progressing, though noticeable pauses may affect the flow. Practicing continuous speech will help reduce hesitations."
    elif fluency_score >= 3.50:
        feedback['fluency'] = "Your fluency shows promise, but frequent hesitations disrupt the flow. Regular practice and confidence-building activities can help improve this."
    else:
        feedback['fluency'] = "Your fluency is developing, but consistent practice will greatly enhance your flow. Don’t hesitate to express yourself confidently!"

    return feedback


feedback = generate_positive_feedback(grammar_score, fluency_score)

print("Feedback:")
print("Grammar:", feedback['grammar'])
print("Fluency:", feedback['fluency'])

NameError: name 'transcription' is not defined

In [ ]:
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
import torchaudio

processor = Wav2Vec2Processor.from_pretrained("Bluecast/wav2vec2-Phoneme")
model = Wav2Vec2ForCTC.from_pretrained("Bluecast/wav2vec2-Phoneme")

audio, rate = torchaudio.load("/content/Recording.wav", normalize=True, channels_first=True)


if audio.shape[0] == 2:
    audio = audio.mean(dim=0, keepdim=True)


target_rate = 16000
if rate != target_rate:
    transform = torchaudio.transforms.Resample(orig_freq=rate, new_freq=target_rate)
    audio = transform(audio)
    rate = target_rate

audio = audio.squeeze(0)

inputs = processor(audio, sampling_rate=rate, return_tensors="pt", padding=True)
logits = model(inputs.input_values).logits
pred_ids = logits.argmax(axis=-1)


phonemes = processor.batch_decode(pred_ids)

if isinstance(phonemes, list):
    phonemes = phonemes[0].split()

print("Phonemes:", phonemes)

phoneme_to_ipa = {
    'aa': 'ɑ',   # Father
    'ae': 'æ',   # Cat
    'ah': 'ʌ',   # Cup
    'ao': 'ɔ',   # Thought
    'aw': 'aʊ',  # House
    'ax': 'ə',   # About
    'ax-h': 'ə', # Ahead (unstressed)
    'axr': 'ɚ',  # Butter (unstressed)
    'ay': 'aɪ',  # My
    'eh': 'ɛ',   # Bed
    'el': 'l̩',  # Syllabic 'l' (Bottle)
    'em': 'm̩',  # Syllabic 'm' (Hymn)
    'en': 'n̩',  # Syllabic 'n' (Button)
    'er': 'ɝ',   # Bird (stressed)
    'ey': 'eɪ',  # They
    'ih': 'ɪ',   # Sit
    'iy': 'i',   # See
    'ow': 'oʊ',  # Go
    'oy': 'ɔɪ',  # Boy
    'uh': 'ʊ',   # Put
    'uw': 'u',   # Blue
    'ux': 'ʉ',   # Goose (Northern American English)
    'b': 'b',    # Bat
    'ch': 'tʃ',  # Cheese
    'd': 'd',    # Dog
    'dh': 'ð',   # This
    'dx': 'ɾ',   # Flap 't' (Butter)
    'eh': 'ɛ',   # Bed
    'f': 'f',    # Fish
    'g': 'g',    # Go
    'hh': 'h',   # Hat
    'jh': 'dʒ',  # Jump
    'k': 'k',    # Cat
    'l': 'l',    # Leg
    'm': 'm',    # Man
    'n': 'n',    # Net
    'ng': 'ŋ',   # Sing
    'p': 'p',    # Pet
    'q': 'ʔ',    # Glottal Stop (Uh-oh)
    'r': 'ɹ',    # Red
    's': 's',    # Sun
    'sh': 'ʃ',   # Shoe
    't': 't',    # Top
    'th': 'θ',   # Thing
    'v': 'v',    # Van
    'w': 'w',    # Wet
    'y': 'j',    # Yes
    'z': 'z',    # Zoo
    'zh': 'ʒ',   # Measure
}


def phonemes_to_ipa(phonemes):
    ipa_transcription = [phoneme_to_ipa.get(phoneme, phoneme) for phoneme in phonemes]
    return ' '.join(ipa_transcription)

ipa_transcription = phonemes_to_ipa(phonemes)
print(f"IPA Transcription: {ipa_transcription}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:86: UserWarning: 
Access to the secret `HF_TOKEN` has not been granted on this notebook.
You will not be requested again.
Please restart the session if you want to be prompted again.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/214 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/298 [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/30.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.34k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

Phonemes: ['hh', 'aw', 't', 'uw', 'ah', 'n', 'ow', 'n', 'd', 'uw', 'ih', 'ng', 'dh', 'ih', 's', 'ay', 't']
IPA Transcription: h aʊ t u ʌ n oʊ n d u ɪ ŋ ð ɪ s aɪ t


In [ ]:
candidate_info = {
    "grammar_score": {grammar_score},
    "fluency_score": {fluency_score},
    "pronunciation_score": 2,
    "candidate_response":{transcription},
    "fluency_feedback": {feedback['fluency']},
    "grammar_feedback": {feedback['grammar']},
    "num_pauses": {num_pauses},
    "filler_words_count": {len(filler_words)},
    "words_per_second": {speaking_rate},
}
prompt = f"""
You are an English language evaluation expert and a hiring manager. You will be provided with the assessment data and feedback from judges for a candidate on grammar, fluency, and pronunciation, along with the candidate's response. Your task is to evaluate the candidate's performance step-by-step, first presenting the judges' feedback as it is, and then explaining and elaborating on it using the provided scores, metrics, and response.

Follow these steps:

1. **Grammar Evaluation**:
   - **Feedback**: Present the feedback for grammar exactly as provided by the judges.
   - **Explanation and Elaboration**: Analyze the candidate's grammar based on the feedback, the score, and their response. Highlight specific strengths (e.g., accurate sentence structures) and weaknesses (e.g., grammatical errors like verb tense mismatches).

2. **Fluency Evaluation**:
   - **Feedback**: Present the feedback for fluency exactly as provided by the judges.
   - **Explanation and Elaboration**: Assess the candidate's fluency using the score and following metrics:
     - Number of filler words (e.g., "um," "uh," "like").
     - Number of noticeable pauses.
     - Words per second (WPS).
   - Provide an analysis of strengths (e.g., smooth delivery) and areas needing improvement (e.g., reducing filler words, increasing pace).

3. **Pronunciation Evaluation**:
   - **Explanation and Elaboration**: Use the pronunciation score to evaluate clarity and articulation, highlighting specific strengths and weaknesses.

Finally, consolidate your analysis into a detailed final report that includes:
- The original judges' feedback for each section.
- The elaboration and explanation for each section.
- Specific, actionable recommendations for improvement.

Feedback from Judges:
- Grammar: {candidate_info['grammar_feedback']}
- Fluency: {candidate_info['fluency_feedback']}

Assessment Scores:
- Grammar: {candidate_info['grammar_score']}/5
- Fluency: {candidate_info['fluency_score']}/5
- Pronunciation: {candidate_info['pronunciation_score']}/5

Additional Metrics:
- Fluency:
  - Number of Filler Words: {candidate_info['filler_words_count']}
  - Number of Pauses: {candidate_info['num_pauses']}
  - Words per Second (WPS): {candidate_info['words_per_second']}

Candidate's Response:
{candidate_info['candidate_response']}

Generate the final report with the structure specified.
"""


response = language_model.generate_content(prompt)
print("Generated Report:")
print(response.text)



NameError: name 'grammar_score' is not defined